# L'évolution de la population scolaire et de la population totale à Paris entre 2019 et 2021, par arrondissement 
### Problématique : Est-ce que l'évolution de la démographie des écoles est en accord avec évolution de la population (par classe d'âge) ?

Claire du Campe de Rosamel, Benoît Nallet, Amandine Perret

## 1. Introduction



Avant de commencer l'analyse on aura besoin des packages suivants : 

In [ ]:
!pip install unidecode
!pip install py7zr geopandas openpyxl tqdm s3fs 
!pip install PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette
!pip install urllib3==1.26.5
!pip install prettytable

In [ ]:
import pandas as pd
import zipfile
import requests
import os
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from unidecode import unidecode 
import matplotlib.ticker as mticker
import geopandas as gpd
from cartiflette import carti_download

Quelques définitions :
Les unités statistiques de notre étude sont :

- la population en logement ordinaire et en résidence principale,
- la population scolaire en école primaire (maternelle et élémentaire).

Il est possible qu'un élève soit scolarisé dans un autre arrondissement que celui où il réside. Il faudra donc distinguer la populaire scolaire dénombrée au niveau de son lieu de résidence principale de celle dénombrée au niveau de son lieu de scolarisation.

Le champ de l'étude est restreint à la ville de Paris pour les années 2019 à 2021.

## 2. Collecte et nettoyage des données 

Notre analyse se fait à partir des plusieurs bases de données publique issues de la Statistique publique et disponible directement sur le site internet de l'Insee.
 
Les effectifs scolaires sont disponibles à l'url XX recensant pour toute la France le nombre d'élèves par classe et par école. Nous avons donc réduit l'étendue de la base de données aux seules écoles dans Paris intra-muros (dont le code postal commence par 75). 

Une deuxième partie des données est issue du recensement fait par l'Insee. Plusieurs bases de données ont été nécessaires : la base logement pour comptabiliser la population habitant effectivement dans chaque arrondissement.

Enfin la base XY renseigne les effectifs de population par tranche d'âge.

Les différentes bases de données sont importées directement depuis leur adresse internet et sont ensuite nettoyées dans le script "import_data.py" ci-dessous.

In [ ]:
from scripts import import_data

## 3. Quels constats /!\ A la fin il faudra enlever les print du script

Depuis la crise sanitaire la ville de Paris a perdu des habitants dans un contexte de vieillissement de la population.

Ces pertes sont concentrées dans le Nord-Est parisien et on retrouve la même géographie de perte pour la population par arrondissement de scolarisation.

Pourtant les fermetures de classes/d'écoles se sont concentrées dans XXXZZZ.


In [ ]:
from scripts import stats_descr_recensement

In [ ]:
from scripts import stats_descr_effectif ;

### A. L'évolution de la population à Paris par arrondissement entre 2019 et 2021
#### a. L'évolution globale 

In [ ]:
print("Tableau de l'évolution globale de la population en résidence principale parisienne")
print(stats_descr_recensement.pop_totale)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/carte_evol_population_niveau.png"))
plt.axis("off")
print("attention légende fausse")

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/carte_evol_population_pourcentage.png"))
plt.axis("off")
print("attention légende fausse")


#### b. Focus sur la pyramide des âges 



In [ ]:
from scripts import stats_ages

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/Pyramide_Paris_ecart1921.png"))
plt.title("Evolution de la pyramide des âges entre 2019 et 2021")
plt.axis("off")
print("attention légende fausse")

### B. L'évolution des effectifs scolaires par arrondissement entre 2019 et 2021
#### a. Au global 


In [ ]:
print("Tableau de la baisse du nombre d'élèves")
print(stats_descr_effectif.table_pertes_df)

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/carte_evol_effectifs_niveau.png"))
plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/carte_evol_effectifs_pourcentage.png"))
plt.axis("off")
print("ca serait utile de mettre le chiffre associé à la borne de droite de la légende")


#### b. La population en primaire

#### c. La population en maternelle 

`Pour l'instant mes cellules tournent, les scripts fonctionnent et on peut normalement récupérer les graphiques et des bases de données comme dans les exemples plus haut`

%matplotlib inline permet que quand le script est importé, tous les graphiques ne s'affichent pas

## 4. Modelisation 

Ici les regressions d'Amandine 

## 5. Conclusion
